# Sliding Tile Solver
We'll solve the 5x5 puzzle in 2 main steps:
1. Using a breakdown of simple steps to get the first 3 rows complete.
2. Using a general A* search algorithm to complete the last 2 rows.

## Setup

In [ ]:
import numpy as np

In [ ]:
import heapq

In [ ]:
maze = [[12,8,2,15,5],[13,3,17,9,10],[1,4,11,14,23],[7,21,"X",20,24],[6,16,22,18,19]]  # Edit this
maze = [[c if c != 'X' else 0 for c in row] for row in maze]
maze = np.array(maze)
maze

## Human/Manual Style Solving Steps

In [ ]:
movemap = [
    [' ', 'U', ' '],
    ['L', ' ', 'R'],
    [' ', 'D', ' ']
]
invmovemap = {
    'U': (-1,0),
    'D': (1,0),
    'L': (0,-1),
    'R': (0,1),
}

In [ ]:
def pathfind(start,end,blockers):
    if start == end:
        return ''
    q = [(*start,'')]
    searched = set([start])
    while q:
        newq = []
        for iX,jX,path in q:
            for ip,jp in [(-1,0),(1,0),(0,-1),(0,1)]:
                if (iX+ip,jX+jp) == end:
                    return path+movemap[ip+1][jp+1]
                if (0 <= iX+ip < 5) and (0 <= jX+jp < 5) \
                    and (iX+ip,jX+jp) not in blockers \
                    and (iX+ip,jX+jp) not in searched:
                    newq.append((iX+ip,jX+jp,path+movemap[ip+1][jp+1]))
                    searched.add((iX+ip,jX+jp))
        q = newq
        
def movepiece(X, curr, target, fixed):
    ppath = pathfind(curr, target, fixed)
    moves = ''
    for pmove in ppath:
        pmove = invmovemap[pmove]
        subtarget = (curr[0]+pmove[0], curr[1]+pmove[1])
        moves += pathfind(X,subtarget, fixed | {curr})
        moves += movemap[1-pmove[0]][1-pmove[1]]
        X = curr
        curr = subtarget
    return moves, X
          
def applymoves(board, moves):
    iX,jX = np.stack(np.where(board == 0)).ravel().tolist()
    for move in moves:
        ip,jp = invmovemap[move]
        board[iX][jX] = board[iX+ip][jX+jp]
        board[iX+ip][jX+jp] = 0
        iX += ip
        jX += jp
    
def easyrow(board, irow, fixed):
    moves = ''
    X = tuple(np.stack(np.where(board == 0)).ravel().tolist())
    rowfixed = set()
    # first place the first 5-1
    for focus in range(1+5*irow, 1+4+5*irow):
        curr = tuple(np.stack(np.where(board == focus)).ravel().tolist())
        target = ((focus-1)//5, (focus-1)%5)
        submoves,X = movepiece(X, curr, target, fixed | rowfixed)
        applymoves(board, submoves)
        moves += submoves
        rowfixed.add(target)
    # last one
    focus = 5*(irow+1)
    curr = tuple(np.stack(np.where(board == focus)).ravel().tolist())
    if curr[0] == irow: #already done
        rowfixed = set()
        for ifocus in range(1,focus+1):
            rowfixed.add(tuple(np.stack(np.where(board == ifocus)).ravel().tolist()))
        fixed |= rowfixed
        return moves
    if X[0] == irow and X[1] == 5-1 and curr[0] == irow+1 and curr[1] == 5-1: #adjacent
        applymoves(board, ['D'])
        moves += 'D'
        rowfixed = set()
        for ifocus in range(1,focus+1):
            rowfixed.add(tuple(np.stack(np.where(board == ifocus)).ravel().tolist()))
        fixed |= rowfixed
        return moves
    if curr[0] > irow+1: # move it up
        submoves,X = movepiece(X, curr, (irow+1, 2), fixed | rowfixed)
        applymoves(board, submoves)
        moves += submoves
        # set X to corner
        rowfixed.add((irow+1, 2))
    submoves = pathfind(X,(irow,5-1),fixed | rowfixed)
    applymoves(board, submoves)
    moves += submoves
    #cycle to ready
    curr = tuple(np.stack(np.where(board == focus)).ravel().tolist())
    currp = tuple(np.stack(np.where(board == focus-1)).ravel().tolist())
    while curr[1] <= currp[1]:
        submoves = 'D'+'L'*4+'U'+'R'*4
        applymoves(board, submoves)
        moves += submoves
        curr = tuple(np.stack(np.where(board == focus)).ravel().tolist())
        currp = tuple(np.stack(np.where(board == focus-1)).ravel().tolist())
    # move to position
    rowfixed = set()
    for ifocus in range(1,focus):
        rowfixed.add(tuple(np.stack(np.where(board == ifocus)).ravel().tolist()))
    submoves,X = movepiece((irow,5-1), curr, (irow,currp[1]+1), fixed | rowfixed)
    applymoves(board, submoves)
    moves += submoves
    rowfixed.add((irow,currp[1]+1))
    # reset X
    submoves = pathfind(X,(irow+1,5-1),fixed | rowfixed)
    applymoves(board, submoves)
    moves += submoves
    # uncycle
    curr = tuple(np.stack(np.where(board == focus)).ravel().tolist())
    while curr[1] < 4:
        submoves = 'U'+'L'*4+'D'+'R'*4
        applymoves(board, submoves)
        moves += submoves
        curr = tuple(np.stack(np.where(board == focus)).ravel().tolist())
    # update fixed
    rowfixed = set()
    for ifocus in range(1,focus+1):
        rowfixed.add(tuple(np.stack(np.where(board == ifocus)).ravel().tolist()))
    fixed |= rowfixed
    return moves

def easyfirstrows(board):
    fixed = set()
    moves = ''
    for i in range(5-2):
        moves += easyrow(board,i,fixed)
    return moves

In [ ]:
moves = easyfirstrows(maze.copy())
print(len(moves))
print(','.join(moves))

In [ ]:
close_maze = maze.copy()
applymoves(close_maze, moves)
close_maze

## A* Algorithm

In [ ]:
target = np.arange(5*5).reshape((5,5))+1
target %= 5*5
target

In [ ]:
targethash = hash(target.tobytes())

In [ ]:
def dist(m):
    mnorm = (m+24)%25
    tnorm = (target+24)%25
    drows = np.abs(mnorm//5-tnorm//5).sum()
    dcols = np.abs(mnorm%5-tnorm%5).sum()
    return drows + dcols

In [ ]:
print(close_maze)
dist(close_maze)

In [ ]:
iX = np.where(close_maze == 0)[0][0]
jX = np.where(close_maze == 0)[1][0]
iX,jX

In [ ]:
step = 0
q = [(dist(close_maze),0,step,iX,jX,'',close_maze)]
searched = set([hash(close_maze.tobytes())])

In [ ]:
final_result = ''
while q and step < 4000000:
    cost,prevd,_,iX,jX,path,curr = heapq.heappop(q)
    for ip,jp in [(-1,0),(1,0),(0,-1),(0,1)]:
        if (3 <= iX+ip < 5) and (0 <= jX+jp < 5):
            curr[iX][jX] = curr[iX+ip][jX+jp]
            curr[iX+ip][jX+jp] = 0
            hashv = hash(curr.tobytes())
            if hashv == targethash:
                final_result = path + movemap[ip+1][jp+1]
                q = []
                break
            if hashv not in searched:
                step += 1
                heapq.heappush(q, (prevd+1+dist(curr), prevd+1, step, iX+ip, jX+jp, path + movemap[ip+1][jp+1], curr.copy()))
                searched.add(hashv)
            curr[iX+ip][jX+jp] = curr[iX][jX]
            curr[iX][jX] = 0
print(final_result)

## Final Result

In [ ]:
final_moves = moves+final_result
print(len(final_moves))
print(','.join(final_moves))

In [ ]:
final_board = maze.copy()
applymoves(final_board, final_moves)
final_board